In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Leer el dataset
df = pd.read_csv('movie_statistic_dataset.csv')

# Analizar los datos
df.head()


,movie_title,production_date,genres,runtime_minutes,director_name,director_professions,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $
0,Avatar: The Way of Water,2022-12-09,"Action,Adventure,Fantasy",192.0,James Cameron,"writer,producer,director",1954,alive,7.8,277543.0,7.061101,460000000,667830256,2265935552
1,Avengers: Endgame,2019-04-23,"Action,Adventure,Drama",181.0,-,-,-,-,8.4,1143642.0,8.489533,400000000,858373000,2794731755
2,Pirates of the Caribbean: On Stranger Tides,2011-05-20,"Action,Adventure,Fantasy",137.0,Rob Marshall,"director,miscellaneous,producer",1960,alive,6.6,533763.0,6.272064,379000000,241071802,1045713802
3,Avengers: Age of Ultron,2015-04-22,"Action,Adventure,Sci-Fi",141.0,Joss Whedon,"writer,producer,director",1964,alive,7.3,870573.0,7.214013,365000000,459005868,1395316979
4,Avengers: Infinity War,2018-04-25,"Action,Adventure,Sci-Fi",149.0,-,-,-,-,8.4,1091968.0,8.460958,300000000,678815482,2048359754


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4380 entries, 0 to 4379
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_title           4380 non-null   object 
 1   production_date       4380 non-null   object 
 2   genres                4380 non-null   object 
 3   runtime_minutes       4380 non-null   float64
 4   director_name         4380 non-null   object 
 5   director_professions  4380 non-null   object 
 6   director_birthYear    4380 non-null   object 
 7   director_deathYear    4380 non-null   object 
 8   movie_averageRating   4380 non-null   float64
 9   movie_numerOfVotes    4380 non-null   float64
 10  approval_Index        4380 non-null   float64
 11  Production budget $   4380 non-null   int64  
 12  Domestic gross $      4380 non-null   int64  
 13  Worldwide gross $     4380 non-null   int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 479.2+ KB


In [11]:
# Seleccionar características y objetivo
features = ['runtime_minutes', 'movie_averageRating', 'movie_numerOfVotes', 'approval_Index', 'Production budget $', 'Domestic gross $', 'genres']
target = 'Worldwide gross $'

X = df[features]
y = df[target]

# Preprocesamiento de datos
numeric_features = ['runtime_minutes', 'movie_averageRating', 'movie_numerOfVotes', 'approval_Index', 'Production budget $', 'Domestic gross $']
categorical_features = ['genres']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear y dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar la transformación
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [ ]:


# Modelo 1
model1 = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model1.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Modelo 2
model2 = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='tanh', kernel_regularizer=l2(0.01)),
    Dense(64, activation='tanh', kernel_regularizer=l2(0.01)),
    Dense(1, activation='linear')
])
model2.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Modelo 3
model3 = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='linear')
])
model3.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Entrenar los modelos
history1 = model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)
history2 = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)
history3 = model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Evaluar los modelos
loss1, mae1 = model1.evaluate(X_test, y_test)
loss2, mae2 = model2.evaluate(X_test, y_test)
loss3, mae3 = model3.evaluate(X_test, y_test)

print(f'Model 1 - Loss: {loss1}, MAE: {mae1}')
print(f'Model 2 - Loss: {loss2}, MAE: {mae2}')
print(f'Model 3 - Loss: {loss3}, MAE: {mae3}')


/home/arg/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-04 19:20:48.046764: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-04 19:20:48.047333: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required

Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 48032152574492672.0000 - mae: 111822328.0000 - val_loss: 61200127167037440.0000 - val_mae: 127529584.0000
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54226010811400192.0000 - mae: 113307760.0000 - val_loss: 61200075627429888.0000 - val_mae: 127529496.0000
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50663584547471360.0000 - mae: 110287216.0000 - val_loss: 61199895238803456.0000 - val_mae: 127529232.0000
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 46641244595552256.0000 - mae: 109031464.0000 - val_loss: 61199534461550592.0000 - val_mae: 127528688.0000
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 62313670452969472.0000 - mae: 117825048.0000 - val_loss: 61198937461096448.0000 - val_mae: 127527864.0000
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 45459488639025152.0000 - mae: 108922088.0000 - val_loss: 61198014043127808.0000 - val_mae: 127526568.0000
Epoc